In [ ]:
# the utils we'll be looking for

# we created groups
# we could calculate intersections
# we could infill groups with -1

# where is my interpolate and combine?

# the interval type seems realy handy as a multiindex with a float value field
    # multi type sleep data
    # single type data present/ not present  for various sensors
    # action intervals with various string descriptors

# is there a way for me to have multiple kinds of values for the value segment?

# start with the coded int intervals
    #how to keep the key? - let's just do it in the docs for now
# then extend to the action intervals

# how about the key for the coded int is the column name
#name+int!....

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import os
import sys

cwd = os.getcwd()
delimiter = "\\" if "\\" in cwd else "/"
repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
sys.path.append(repoPath + "dataImport/")
import rwWorkingTSDf
from rwWorkingTSDf import readWorkingTSDF, writeWorkingTSDf


In [ ]:
def infillIntervals(intervals):
    # Shift columns to compare consecutive rows
    next_start = intervals['startTime'].shift(-1)

    # Find rows where there is a gap
    gap_mask = next_start > intervals['endTime']

    # Create a DataFrame for the gaps
    gap_df = pd.DataFrame({
        'startTime': intervals.loc[gap_mask, 'endTime'],
        'endTime': next_start[gap_mask],
        'value': 0
    })

    # Concatenate the original DataFrame and the gap DataFrame
    combined_df = pd.concat([intervals, gap_df])

    return combined_df.sort_values('startTime').reset_index(drop=True)

In [ ]:
#modify the dd datatype to append "-dataIntervals-86400"
# in seconds with p for .


# so something we can do is make intervals for existing data
# min and max interval are in seconds
def samplesToIntervals(dd, max_interval):
    #take in a device descriptor and min and max interval in seconds?
    #column name would be noData+0!data+1
    tsDf = readWorkingTSDF(dd)

    time_diffs = tsDf.index.to_series().diff()

    # Mark where the time difference exceeds the threshold (group break points)
    group_breaks = time_diffs.dt.total_seconds() > max_interval

    # Assign group labels by cumulatively summing the group breaks
    group_labels = group_breaks.cumsum()

    # Create a dataframe of (start_time, end_time) for each group
    group_labels_times = pd.Series(group_labels.index, index=group_labels.values)
    gStarts = group_labels_times[~group_labels_times.index.duplicated(keep="first")]
    gEnds = group_labels_times[~group_labels_times.index.duplicated(keep="last")]
    allGroupsDf = pd.DataFrame({'sampleDT':gStarts, 'endTime': gEnds})

    #check min group time
    minGroupTime = pd.Timedelta(seconds=0)
    groupsDf = allGroupsDf[allGroupsDf['endTime'] - allGroupsDf['sampleDT'] > minGroupTime].copy()

    
    groupsDf['int!noInterval+0!interval+1'] = 1
    return infillIntervals(groupsDf)


In [ ]:
#alright new plan for the intervals just use writeWorking TSDF 
# add dedupe = false if needed

In [ ]:
# what did I want to do with these intervals?
# determine where intervals for audio data were so I could query the aligned data struture better for actual audio

# query times when cameras were recording fast, slow, or not at all

# query intersecting HR intervals

# store hypnographs

# find gaps in air quality data

# TODO make separate read and write interval functions

In [ ]:
# after I get my audio imported and intervaled
# pandas allows for non unique indicies 
# just don't deduplicate the data
# we can store the activites in a standard interval 